# MNIST

## Imports

In [26]:
import  torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch import nn
import json
%matplotlib inline

## Inspect Dataset

In [27]:
training_images = MNIST(
    root="data",
    train=True,
    download=True,
)

test_images = MNIST(
    root="data",
    train=False,
    download=True
)

In [28]:
for X, y in training_images:
    print(f"Label: {y}")
    display(X)
    break

Label: 5


## Prepare Data

In [29]:
training_data = MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [30]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [31]:
for X, y in train_dataloader:
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    break

X shape: torch.Size([64, 1, 28, 28])
y shape: torch.Size([64])


## Set Device

In [32]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


## Define Models

### Model with 1 Hidden Layer

In [33]:
class NeuralNet_1_Layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits
    
model_1 = NeuralNet_1_Layer().to(device)
print(model_1)

NeuralNet_1_Layer(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Model with 2 Hidden Layers

In [34]:
class NeuralNet_2_Layers(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits
    
model_2 = NeuralNet_2_Layers().to(device)
print(model_2)

NeuralNet_2_Layers(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Define Train and Test Loops

In [35]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    correct = {}
    loss_dict = {}
    for batch, (X, y) in enumerate(dataloader):
        X, y  = X.to(device), y.to(device)
        batch_size = X.shape[0]

        pred = model(X)
        loss = loss_fn(pred, y)

        correct[batch + 1] = (pred.argmax(1) == y).type(torch.float).sum().item() / batch_size
        loss_dict[batch + 1] = loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current =  loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} {current:>5d}/{size:>5d}")

    return correct, loss_dict

In [36]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct, test_loss

## Define function to train model

In [38]:
def train_model(model, loss_fn, optimizer, epochs):
    metrics = {
        "train_loss": {},
        "train_accuracy": {},
        "test_loss": {},
        "test_accuracy": {}
    }
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_acc, train_loss = train(train_dataloader, model, loss_fn, optimizer)
        test_acc, test_loss = test(test_dataloader, model, loss_fn)

        metrics["train_accuracy"][f"epoch_{t+1}"] = train_acc
        metrics["train_loss"][f"epoch_{t+1}"] = train_loss
        metrics["test_accuracy"][f"epoch_{t+1}"] = test_acc
        metrics["test_loss"][f"epoch_{t+1}"] = test_loss

    print("Done!")    
    return metrics

## Train Models

In [39]:
num_epochs = 20

### Train model with one hidden layer

In [40]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_1.parameters(), lr=1e-3)
metrics_1_layer = train_model(model_1, loss_fn, optimizer, num_epochs)

Epoch 1
-------------------------------
loss: 2.308230    64/60000
loss: 2.281662  6464/60000
loss: 2.260765 12864/60000
loss: 2.197746 19264/60000
loss: 2.192281 25664/60000
loss: 2.182752 32064/60000
loss: 2.144639 38464/60000
loss: 2.174937 44864/60000
loss: 2.105786 51264/60000
loss: 2.066146 57664/60000
Test Error: 
 Accuracy: 67.6%, Avg loss: 2.066913 

Epoch 2
-------------------------------
loss: 2.074008    64/60000
loss: 2.030106  6464/60000
loss: 2.049876 12864/60000
loss: 1.905461 19264/60000
loss: 1.931696 25664/60000
loss: 1.922265 32064/60000
loss: 1.854143 38464/60000
loss: 1.942520 44864/60000
loss: 1.819878 51264/60000
loss: 1.772315 57664/60000
Test Error: 
 Accuracy: 74.8%, Avg loss: 1.765025 

Epoch 3
-------------------------------
loss: 1.782826    64/60000
loss: 1.708288  6464/60000
loss: 1.761524 12864/60000
loss: 1.562385 19264/60000
loss: 1.597979 25664/60000
loss: 1.590347 32064/60000
loss: 1.505867 38464/60000
loss: 1.654161 44864/60000
loss: 1.494347 51264

In [41]:
with open("results/one_layer_nn_metrics.json", "w") as f:
    json.dump(metrics_1_layer, f)

### Train model with two hidden layers

In [42]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_2.parameters(), lr=1e-3)
metrics_2_layer = train_model(model_2, loss_fn, optimizer, num_epochs)

Epoch 1
-------------------------------
loss: 2.306027    64/60000
loss: 2.298787  6464/60000
loss: 2.289389 12864/60000
loss: 2.284645 19264/60000
loss: 2.288218 25664/60000
loss: 2.279191 32064/60000
loss: 2.271676 38464/60000
loss: 2.275934 44864/60000
loss: 2.264860 51264/60000
loss: 2.249436 57664/60000
Test Error: 
 Accuracy: 41.5%, Avg loss: 2.255874 

Epoch 2
-------------------------------
loss: 2.255110    64/60000
loss: 2.246087  6464/60000
loss: 2.247799 12864/60000
loss: 2.217649 19264/60000
loss: 2.235837 25664/60000
loss: 2.225721 32064/60000
loss: 2.206549 38464/60000
loss: 2.226870 44864/60000
loss: 2.198202 51264/60000
loss: 2.179139 57664/60000
Test Error: 
 Accuracy: 60.1%, Avg loss: 2.185658 

Epoch 3
-------------------------------
loss: 2.183012    64/60000
loss: 2.169458  6464/60000
loss: 2.183993 12864/60000
loss: 2.117229 19264/60000
loss: 2.152824 25664/60000
loss: 2.139756 32064/60000
loss: 2.102050 38464/60000
loss: 2.142637 44864/60000
loss: 2.088901 51264

In [43]:
with open("results/two_layer_nn_metrics.json", "w") as f:
    json.dump(metrics_2_layer, f)